In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
journals_contracts = pd.read_pickle('../data/pickle/'+cfg['journal_contracts_file']+'.pickle')

df = journals_contracts
del journals_contracts

df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-12-31,DLT1,2015-01-01,0.00,91.31,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
1,2015-12-31,DLT1,2015-01-01,0.00,182.62,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
2,2015-12-31,DLT1,2015-01-01,0.00,43.88,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY ONLINE (PHOENIX),HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
3,2015-12-31,DLF1,2015-01-01,0.00,445.15,A&G_MAEU,NaN,Renewal,Netherlands,A&G,...,EUROPEAN GERIATRIC MEDICINE,AG-MAEU-North_Europe0-01D,LF,N.A,0-00-00,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED
4,2015-12-31,DLF1,2015-01-01,0.00,183.47,A&G_MAEU,NaN,Renewal,Netherlands,Global,...,ANNALES DE DERMATOLOGIE,AG-MAEU-North_Europe0-01D,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED


# Replace strings indicating mssing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00'], np.nan, inplace=True)
df

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-12-31,DLT1,2015-01-01,0.00,91.31,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
1,2015-12-31,DLT1,2015-01-01,0.00,182.62,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
2,2015-12-31,DLT1,2015-01-01,0.00,43.88,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY ONLINE (PHOENIX),HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
3,2015-12-31,DLF1,2015-01-01,0.00,445.15,A&G_MAEU,NaN,Renewal,Netherlands,A&G,...,EUROPEAN GERIATRIC MEDICINE,AG-MAEU-North_Europe0-01D,LF,NaN,NaN,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED
4,2015-12-31,DLF1,2015-01-01,0.00,183.47,A&G_MAEU,NaN,Renewal,Netherlands,Global,...,ANNALES DE DERMATOLOGIE,AG-MAEU-North_Europe0-01D,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144117,2020-12-31,1-17874004774,2016-01-01,0.00,7194.61,A&G_NOAM,Cancelled,Renewal,United States,A&G,...,Veterinary Parasitology,AG-NOAM-CAN-NEW-WEST0-18D,Standard Renewal,Approved,2019-12-30,Region Canada New West,2020-12-31,2020-01-01,2020,INVOICED
4144118,2020-12-31,1-17874004774,2016-01-01,0.00,7770.80,A&G_NOAM,Cancelled,Renewal,United States,A&G,...,Virus Research,AG-NOAM-CAN-NEW-WEST0-18D,Standard Renewal,Approved,2019-12-30,Region Canada New West,2020-12-31,2020-01-01,2020,INVOICED
4144119,2020-12-31,1-17874004774,2016-01-01,0.00,11140.77,A&G_NOAM,Cancelled,Renewal,United States,A&G,...,Water Research,AG-NOAM-CAN-NEW-WEST0-18D,Standard Renewal,Approved,2019-12-30,Region Canada New West,2020-12-31,2020-01-01,2020,INVOICED
4144120,2020-12-31,1-17874004774,2016-01-01,0.00,612.45,A&G_NOAM,Cancelled,Renewal,United States,A&G,...,Women's Health Issues,AG-NOAM-CAN-NEW-WEST0-18D,Standard Renewal,Approved,2019-12-30,Region Canada New West,2020-12-31,2020-01-01,2020,INVOICED


### Summary of the dataframe

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
24,Renewal Exp Complete Date,"[[nan, 2015-12-20, 2015-11-15, 2015-09-19, 201...",1669,object,43.58
16,Payment Term Description,"[[nan, Due 31 January 2015, Net Due in 30 days...",101,object,36.56
17,Payment Term Type,"[[nan, AS_IS_DATE, NET, SPL/MTH]]",4,object,36.47
15,Payment Term,"[[nan, 31JAN15, 30 NET, 15DEC14, 60 NET, 31MAR...",104,object,36.47
14,Parent Agreement Number,"[[nan, 1-5347705377, 1-5302903067, 1-598483264...",69021,object,35.42
30,Status Change Date,"[[nan, 2014-12-20, 2015-07-29, 2014-09-16, 201...",1506,object,33.82
29,Status,"[[nan, Complete, Merged Complete, With Custome...",16,object,33.81
6,Business Indicator,"[[nan, Renewal, New Sale, Cancelled, Credit, N...",6,object,33.31
28,Sales Type,"[[MS, LF, GR, LC, nan, MD, PF, VS, PC, OT, SN,...",23,object,7.25
11,Invoice Date,"[[2015-01-08, 2015-01-14, 2014-11-18, 2014-12-...",1862,object,2.54


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In this case, manually ignore Renewal Exp Complete Date.

In [5]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove borderline case
df = df.drop(columns='Renewal Exp Complete Date')

-------------------- Dropping Columns with missing data --------------------
Following 0 columns have missing data over the threshold and will be removed
Index([], dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
16,Payment Term Description,"[[nan, Due 31 January 2015, Net Due in 30 days...",101,object,36.56
17,Payment Term Type,"[[nan, AS_IS_DATE, NET, SPL/MTH]]",4,object,36.47
15,Payment Term,"[[nan, 31JAN15, 30 NET, 15DEC14, 60 NET, 31MAR...",104,object,36.47
14,Parent Agreement Number,"[[nan, 1-5347705377, 1-5302903067, 1-598483264...",69021,object,35.42
29,Status Change Date,"[[nan, 2014-12-20, 2015-07-29, 2014-09-16, 201...",1506,object,33.82
28,Status,"[[nan, Complete, Merged Complete, With Custome...",16,object,33.81
6,Business Indicator,"[[nan, Renewal, New Sale, Cancelled, Credit, N...",6,object,33.31
27,Sales Type,"[[MS, LF, GR, LC, nan, MD, PF, VS, PC, OT, SN,...",23,object,7.25
11,Invoice Date,"[[2015-01-08, 2015-01-14, 2014-11-18, 2014-12-...",1862,object,2.54
12,Invoice Num,"[[A2448269, A2448270, A2449813, 3492988Y, 3542...",1169678,object,2.54


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

*Parent Agreement Number* - New Binary Categorial Variable "Has_Parent" 

Payment Term Type - Impute with most frequent (Check with Business")

Business Indicator - Impute with most frequent

Sales Type - Impute with most frequent

Bookings - Final Net Price - Agent Discount Am - Impute with median

In [7]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['Payment Term Type'].fillna(
#     df['Payment Term Type'].value_counts().idxmax(), inplace=True)

# df['Business Indicator'].fillna(
#     df['Business Indicator'].value_counts().idxmax(), inplace=True)

# df['Sales Type'].fillna(
#     df['Sales Type'].value_counts().idxmax(), inplace=True)

# # Filling missing values with median value
# df['Bookings - Final Net Price - Agent Discount Amount(Rep)'].fillna(
#     df['Bookings - Final Net Price - Agent Discount Amount(Rep)'].median(),
#     inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [8]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['Bookigns - Committed Print(Rep)', 'Subscription Start Year'])

Dropping outliers for Bookings - Final Net Price - Agent Discount Amount(Rep) upper limit = 8611.897398242927 and lower limit = -221.80949999999984


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [9]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
18,Product Line Level 1,[[Journals]],1,object,0.00
22,Product Revenue Type,"[[Recurring, One Off]]",2,object,0.00
7,Calculated New/Renewal,"[[Renewal, New]]",2,object,0.00
17,Payment Term Type,"[[nan, AS_IS_DATE, NET, SPL/MTH]]",4,object,39.37
9,Division,"[[Global, A&G, HS, Corporate, nan]]",5,object,0.00
34,WIP Flag,"[[INVOICED, MERGED, WIP_PPL, WIP, DELETED]]",5,object,0.00
6,Business Indicator,"[[nan, Renewal, New Sale, Cancelled, Credit, N...",6,object,36.11
33,Subscription Start Year,"[[2015, 2016, 2017, 2018, 2019, 2020]]",6,int64,0.00
5,Business Division (Agreement SIS),"[[HS EMEA/LA, A&G_MAEU, A&G_NOAM, Corporate, A...",9,object,0.00
23,RSO,"[[Member, MAEU, NOAM, Personal, EMEALA, APAC, ...",10,object,0.00


In [11]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [12]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [13]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
18,Product Line Level 1,[[Journals]],1,object,0.00
22,Product Revenue Type,"[[Recurring, One Off]]",2,object,0.00
7,Calculated New/Renewal,"[[Renewal, New]]",2,object,0.00
17,Payment Term Type,"[[nan, AS_IS_DATE, NET, SPL/MTH]]",4,object,39.37
9,Division,"[[Global, A&G, HS, Corporate, nan]]",5,object,0.00
34,WIP Flag,"[[INVOICED, MERGED, WIP_PPL, WIP, DELETED]]",5,object,0.00
6,Business Indicator,"[[nan, Renewal, New Sale, Cancelled, Credit, N...",6,object,36.11
33,Subscription Start Year,"[[2015, 2016, 2017, 2018, 2019, 2020]]",6,int64,0.00
5,Business Division (Agreement SIS),"[[HS EMEA/LA, A&G_MAEU, A&G_NOAM, Corporate, A...",9,object,0.00
23,RSO,"[[Member, MAEU, NOAM, Personal, EMEALA, APAC, ...",10,object,0.00


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [14]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [15]:
df = scale_numeric_features(df, exclude=['Subscription Start Year'], inplace=True)

********************* - Scaling following 2 features - **********************
 Bookigns - Committed Print(Rep) standardized to Bookigns - Committed Print(Rep)
 Bookings - Final Net Price - Agent Discount Amount(Rep) standardized to Bookings - Final Net Price - Agent Discount Amount(Rep)


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [16]:
#print(columns_to_bin)

In [17]:
# Specify Columns to encode
# columns_to_exclude = ['Product Line Level 3','Payment Term Description','Payment Term', 
#                       'Business Division (Agreement SIS)', 'Product Line Level 4']
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [18]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [19]:
#df.shape

In [20]:
df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-12-31,DLT1,2015-01-01,-0.11,-0.78,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
1,2015-12-31,DLT1,2015-01-01,-0.11,-0.73,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
2,2015-12-31,DLT1,2015-01-01,-0.11,-0.81,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY ONLINE (PHOENIX),HS Europe North,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED
3,2015-12-31,DLF1,2015-01-01,-0.11,-0.60,A&G_MAEU,NaN,Renewal,Netherlands,A&G,...,EUROPEAN GERIATRIC MEDICINE,AG-MAEU-North_Europe0-01D,LF,NaN,NaN,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED
4,2015-12-31,DLF1,2015-01-01,-0.11,-0.73,A&G_MAEU,NaN,Renewal,Netherlands,Global,...,ANNALES DE DERMATOLOGIE,AG-MAEU-North_Europe0-01D,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [21]:
# convert date columns to date data type

date_columns = ['Agreement Start Date', 'Agreement End Date','Subscription Start Date', 'Subscription End Date']

In [22]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [23]:
# calcualte new variable subscription_length and agreement_length

In [24]:
df['subscription_length'] = months_between_columns(df['Subscription End Date'],df['Subscription Start Date'])

In [25]:
df['agreement_length'] = months_between_columns(df['Agreement End Date'],df['Agreement Start Date'])

In [26]:
 df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag,subscription_length,agreement_length
0,2015-12-31,DLT1,2015-01-01,-0.11,-0.78,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
1,2015-12-31,DLT1,2015-01-01,-0.11,-0.73,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
2,2015-12-31,DLT1,2015-01-01,-0.11,-0.81,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
3,2015-12-31,DLF1,2015-01-01,-0.11,-0.60,A&G_MAEU,NaN,Renewal,Netherlands,A&G,...,LF,NaN,NaN,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
4,2015-12-31,DLF1,2015-01-01,-0.11,-0.73,A&G_MAEU,NaN,Renewal,Netherlands,Global,...,MS,NaN,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [27]:
# @TODO
# check the columns use of 'Status Change Date', 'Renewal Exp Complete Date'
# check 'Bookigns - Committed Print(Rep)'

columns_to_remove = ['HQ SIS Id (Agreement SIS)', 'Name  (Agreement SIS)', 'Invoice Num', 'Payment Term Description', 
                     'Status Change Date', 'Product Line Level 3', 'Product Line Level 4',
                     'Bookigns - Committed Print(Rep)']

In [28]:
df = df.drop(columns_to_remove, axis=1)

In [29]:
df.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,Invoice Date,...,Sales Division (Agreement SIS),Sales Type,Status,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag,subscription_length,agreement_length
0,2015-12-31,DLT1,2015-01-01,-0.78,HS EMEA/LA,NaN,Renewal,Netherlands,Global,2015-01-08,...,HS Europe North,MS,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
1,2015-12-31,DLT1,2015-01-01,-0.73,HS EMEA/LA,NaN,Renewal,Netherlands,Global,2015-01-08,...,HS Europe North,MS,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
2,2015-12-31,DLT1,2015-01-01,-0.81,HS EMEA/LA,NaN,Renewal,Netherlands,Global,2015-01-14,...,HS Europe North,MS,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
3,2015-12-31,DLF1,2015-01-01,-0.60,A&G_MAEU,NaN,Renewal,Netherlands,A&G,2014-11-18,...,AG-MAEU-North_Europe0-01D,LF,NaN,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00
4,2015-12-31,DLF1,2015-01-01,-0.73,A&G_MAEU,NaN,Renewal,Netherlands,Global,2014-12-12,...,AG-MAEU-North_Europe0-01D,MS,NaN,Global Member,2015-12-31,2015-01-01,2015,INVOICED,12.00,12.00


Save clean pickle files

In [30]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['journal_contracts_file']+"_for_graphs.pickle")